In [1]:
import pandas as pd
import nltk
import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize
from razdel import tokenize, sentenize
from navec import Navec
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from slovnet.model.emb import NavecEmbedding
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import cupy as cp
import pickle 
import scipy.sparse as sp

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from navec import Navec
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, MorphVocab, Doc 

In [4]:
# импортируем библиотеку pandarallel и инициализируем её
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=6)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [5]:

navec = Navec.load('E:/mts/navec_hudlit_v1_12B_500K_300d_100q.tar')


# morph_vocab = MorphVocab()

# segmenter = Segmenter()
# emb = NewsEmbedding()
# morph_tagger = NewsMorphTagger(emb)
# syntax_parser = NewsSyntaxParser(emb)


In [6]:
df = pd.read_csv("E:/mts/scrap/urls_info_0.csv")
df['url_info'] = df['url_info'].fillna(' ')

In [7]:
def get_emb(text):
    
    from navec import Navec
    from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, MorphVocab, Doc 
    import numpy as np
    from navec import Navec
    
    navec = Navec.load('E:/mts/navec_hudlit_v1_12B_500K_300d_100q.tar')
    
    morph_vocab = MorphVocab()

    segmenter = Segmenter()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)  
    
    if text == ' ':
        return np.zeros((1, 300))[0]
    
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        
    lemantized_text = [token.lemma for token in doc.tokens]
    
    lenght = len(lemantized_text)
    emb = np.zeros((1, 300))
    for lemma in lemantized_text:
        
        try:
            emb += navec[lemma]
        except:
            pass
            
    emb = np.mean(emb, axis=0)
    
    
    return emb
    

In [8]:
%%script false
%%time
embs = df['url_info'].parallel_apply(get_emb)


df_embs = pd.concat([df['url_host'], pd.DataFrame(embs.to_list())], axis=1)
df_embs.columns = list(map(str, df_embs.columns))
df_embs.reset_index(inplace=True)
df_embs.drop(['index'], axis=1, inplace=True)
df_embs.to_feather('E:/mts/scrap/embs.feather')

Couldn't find program: 'false'


In [9]:
%%script false
%%time
df_for_urls = pd.read_feather('E:/mts/total.feather')[['user_id', 'url_host']].groupby('user_id')['url_host'].apply(list).apply(np.array)

df_embs = pd.read_feather('E:/mts/scrap/embs.feather')
df_embs.set_index('url_host', inplace=True)
dict_url_emb = {index: np.array(row) for index, row in df_embs.iterrows()}

Couldn't find program: 'false'


In [10]:
def get_one_emb(lst):
    
    import numpy as np
    
    
    res = np.zeros((1, 300))[0]
    count = 0
    for url in lst:
        try:
            matches = dict_url_emb[url] #url_embs[url_embs['url_host'] == url].drop('url_host', axis=1).astype('float').values
        except:
            count -= 1
            continue
        res += np.round(matches, 5) # add the embedding to the result (excluding the first column)
        count += 1
    if count == 0: # no embeddings added to the result
        return None
    return np.round(res / count, 5)


In [11]:
%%script false
%%time 
# one embedding of all visited sites for each user
emb_all_sites = df_for_urls.apply(get_one_emb)
emb_all_sites = pd.DataFrame(emb_all_sites)

Couldn't find program: 'false'


In [12]:
%%script false
emb_all_sites = emb_all_sites.join(emb_all_sites['url_host'].apply(pd.Series).add_prefix('host_'))
emb_all_sites = emb_all_sites.drop('url_host', axis=1)
emb_all_sites.reset_index(inplace=True)
emb_all_sites.to_feather('E:/mts/user_emb_host.feather')

Couldn't find program: 'false'


In [13]:
# pd.read_feather('E:/mts/user_emb_host.feather')

# сделать TF-IDF для названий сайтов, fit на обучающей transform на предсказываемых

# параметр колво посещенных сайтов на количество посещений всех сайтов

In [14]:
# df_for_urls = pd.read_feather('E:/mts/total.feather')[['user_id', 'url_host']].groupby('user_id')['url_host'].apply(list).apply(np.array)
# df_for_urls = {idx: ', '.join(df_for_urls[idx])  for idx in df_for_urls.index}

In [15]:
%%script false

df_for_urls = pd.read_feather('E:/mts/total.feather')[['user_id', 'url_host','request_cnt', 'date']]
df_for_urls = df_for_urls.sort_values(by='date')

Couldn't find program: 'false'


In [16]:
%%script false

# Оформляем историю каждого пользователя во времени по количеству посещений каждого сайта
user_data = {}
for _, d in tqdm.tqdm(df_for_urls.iterrows(), total = len(df_for_urls)):
    if d['user_id'] not in user_data:
        user_data[d['user_id']] = []
    for i in range(d['request_cnt']):
        user_data[d['user_id']].append(d['url_host'])

# print(user_data)

Couldn't find program: 'false'


In [17]:
%%script false
user_data = dict(sorted(user_data.items()))

with open ('E:/mts/dict_user_ordered_urls_visit.pkl', 'wb') as f:
    pickle.dump(user_data, f)

Couldn't find program: 'false'


In [18]:
# %%script false
with open ('E:/mts/dict_user_ordered_urls_visit.pkl', 'rb') as f:
    user_data = pickle.load(f)  
    
user_data = {idx: ', '.join(user_data[idx])  for idx in user_data.keys()}

test_users = pd.read_parquet('E:/mts/submit_2.pqt')['user_id'].values
train_users = pd.read_parquet('E:/mts/public_train.pqt')['user_id'].values


train_tf_idf_users = {k: v for k, v in user_data.items() if k  in train_users}
test_tf_idf_users = {k: v for k, v in user_data.items() if k  in test_users}

In [19]:
del user_data, test_users, train_users

In [20]:
# %%script false
pattern = r'\b(?:[a-z0-9](?:[a-z0-9\-]{0,61}[a-z0-9])?\.)+(?:[a-z]{2,6}\.?|[a-z0-9\-]{2,}\.?)\S*\b'
vectorizer = TfidfVectorizer(token_pattern=pattern, dtype=cp.float32, ngram_range=(1, 5), max_df=0.2, min_df=0, max_features=3000)
matrix_train = vectorizer.fit_transform(train_tf_idf_users.values())
matrix_test = vectorizer.transform(test_tf_idf_users.values())


sp.save_npz('E:/mts/tf_idf_train.npz', matrix_train)
sp.save_npz('E:/mts/tf_idf_test.npz', matrix_test)

In [21]:
# %%script false
pattern = r'\b(?:[a-z0-9](?:[a-z0-9\-]{0,61}[a-z0-9])?\.)+(?:[a-z]{2,6}\.?|[a-z0-9\-]{2,}\.?)\S*\b'
vectorizer = TfidfVectorizer(token_pattern=pattern, dtype=cp.float32, ngram_range=(1, 5), max_df=0.4, min_df=0.2, max_features=3000)
matrix_train = vectorizer.fit_transform(train_tf_idf_users.values())
matrix_test = vectorizer.transform(test_tf_idf_users.values())


sp.save_npz('E:/mts/tf_idf_train_1.npz', matrix_train)
sp.save_npz('E:/mts/tf_idf_test_1.npz', matrix_test)

In [22]:
# %%script false
pattern = r'\b(?:[a-z0-9](?:[a-z0-9\-]{0,61}[a-z0-9])?\.)+(?:[a-z]{2,6}\.?|[a-z0-9\-]{2,}\.?)\S*\b'
vectorizer = TfidfVectorizer(token_pattern=pattern, dtype=cp.float32, ngram_range=(1, 5), max_df=0.6, min_df=0.4, max_features=3000)
matrix_train = vectorizer.fit_transform(train_tf_idf_users.values())
matrix_test = vectorizer.transform(test_tf_idf_users.values())


sp.save_npz('E:/mts/tf_idf_train_2.npz', matrix_train)
sp.save_npz('E:/mts/tf_idf_test_2.npz', matrix_test)

In [23]:
# %%script false
pattern = r'\b(?:[a-z0-9](?:[a-z0-9\-]{0,61}[a-z0-9])?\.)+(?:[a-z]{2,6}\.?|[a-z0-9\-]{2,}\.?)\S*\b'
vectorizer = TfidfVectorizer(token_pattern=pattern, dtype=cp.float32, ngram_range=(1, 5), max_df=0.8, min_df=0.6, max_features=3000)
matrix_train = vectorizer.fit_transform(train_tf_idf_users.values())
matrix_test = vectorizer.transform(test_tf_idf_users.values())


sp.save_npz('E:/mts/tf_idf_train_3.npz', matrix_train)
sp.save_npz('E:/mts/tf_idf_test_3.npz', matrix_test)

In [24]:
# %%script false
pattern = r'\b(?:[a-z0-9](?:[a-z0-9\-]{0,61}[a-z0-9])?\.)+(?:[a-z]{2,6}\.?|[a-z0-9\-]{2,}\.?)\S*\b'
vectorizer = TfidfVectorizer(token_pattern=pattern, dtype=cp.float32, ngram_range=(1, 5), min_df=0.8, max_features=3000)
matrix_train = vectorizer.fit_transform(train_tf_idf_users.values())
matrix_test = vectorizer.transform(test_tf_idf_users.values())


sp.save_npz('E:/mts/tf_idf_train_4.npz', matrix_train)
sp.save_npz('E:/mts/tf_idf_test_4.npz', matrix_test)

#  присобачить tf idf к таргету
# добавить среднее время непоявления/ появления в сети
# добавить tf idf по датам
# если будет ништяк, то поделать и по другим признакам типа устройств

In [25]:
# [agg_time_between_first_last_visit,
#  mean_using_internet,
#  user_embs_usertime_per_part_day,
#  user_embs_url_request_cnt_sum,
#  user_embs_tld_request_cnt_sum,
#  user_embs_cpe_request_cnt_sumб
#  user_embs_part_of_day_request_sum,
#  data_agg_part_of_day_dumm,
#  data_agg_region
# ]

# Tf-IDF по уникальным датам

In [26]:
%%script false

df_for_days = pd.read_feather('E:/mts/total.feather')[['user_id', 'date']]
df_for_days = df_for_days.sort_values(by='date')
df_for_days['date'] = df_for_days['date'].dt.day_of_year.map(str)


df_for_days = dict(df_for_days.groupby('user_id')['date'].unique().apply(lambda x: ', '.join(x)))

Couldn't find program: 'false'


In [27]:
%%script false
with open ('E:/mts/dict_user_ordered_date_visit.pkl', 'wb') as f:
    pickle.dump(df_for_days, f)

Couldn't find program: 'false'


In [28]:
with open ('E:/mts/dict_user_ordered_date_visit.pkl', 'rb') as f:
    date_data = pickle.load(f)

In [29]:
%%script false

test_users = pd.read_parquet('E:/mts/submit_2.pqt')['user_id'].values
train_users = pd.read_parquet('E:/mts/public_train.pqt')['user_id'].values


train_tf_idf_date = {k: v for k, v in date_data.items() if k  in train_users}
test_tf_idf_date = {k: v for k, v in date_data.items() if k  in test_users}

Couldn't find program: 'false'


In [30]:
%%script false
pattern = r'\s*,\s*'
vectorizer = TfidfVectorizer(token_pattern=pattern, dtype=cp.float32, ngram_range=(1, 10), max_features=1000)
matrix_train = vectorizer.fit_transform(train_tf_idf_date.values())
matrix_test = vectorizer.transform(test_tf_idf_date.values())


sp.save_npz('E:/mts/date_tf_idf_train.npz', matrix_train)
sp.save_npz('E:/mts/date_tf_idf_test.npz', matrix_test)

Couldn't find program: 'false'
